In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline

# import sys
# from pathlib import Path
# sys.path.append(str(Path().cwd().parent))

from utils import MinMaxScaler, get_ticket, MakeTimeSeries
from agent import Agent_xgb

from indicators import *

In [ ]:
data = get_ticket("MGLU3.SA")
# data = get_ticket("VALE3.SA")
# data = data[-252:]

In [ ]:
data.head()

In [ ]:
preprocessor = Pipeline(steps=[
    # ("Scaler", MinMaxScaler(cols_list=["_Open", "_High", "_Low", "_Close"])),
    ("SMA", SimpleMovingAverage(short=7, middle=23, long=71, graphic=True)),
    ("RSI/IFR", RelativeStrengthIndex(window=14, graphic=True)),
    ("AO", AwesomeOscillator(short=5, long=34, graphic=True)),
    ("MACD", MovingAverageConvergenceDivergence(short=12, long=26, signal=9, graphic=True)),
    ("ADX/DMI", AverageDirectionalIndex(window=14, threshold=20, graphic=True)),
    # ("OBV", OnBalanceVolume()), # Transformar num sinal!
    ("BB", BollingerBands(window=40, entry_threshold=0.5, out_tp_threshold=1.5, out_sl_threshold=0, graphic=True)),
    ("Ichimoku", IchimokuClouds(window1=9, window2=26, window3=52, graphic=True)),
    # ("VWAP", VolumeWeightedAveragePrice(graphic=True)),  # Transformar num sinal!
    ("Par.SAR", ParabolicSAR(acceleration=0.02, maximum=0.2, graphic=True)),
    ("Aleatory", Aleatory(lack_trend=150, graphic=True)),
    ("Trend", Trend(short=23, long=71, graphic=True)),
    ("MakeTimeSeries", MakeTimeSeries(window=1, indicators=None)) # indicators=['SMA', ...]
])

In [ ]:
_ = preprocessor.fit(data)
data_preprocessed = preprocessor.transform(data)

In [ ]:
data_preprocessed.head()

In [ ]:
sample = data_preprocessed.iloc[-500:].copy()
data_preprocessed.drop(sample.index, inplace=True)

In [ ]:
sample.sample(5)

In [ ]:
despise = ['_Open', '_High', '_Low', '_Close', '_Volume', 'Aleatory D-0']
# despise = ['_Open', '_High', '_Low', '_Close', '_Volume', 'Aleatory D-0', 'Aleatory D-1', 'Aleatory D-2', 'Aleatory D-3', 'Aleatory D-4', 'Aleatory D-5']

In [ ]:
agent = Agent_xgb(data = data_preprocessed, drop_intersection_time_series = None, despise = despise)

### Results Optimum

In [ ]:
agent.optimum_choice(data_preprocessed)

### Naive Choice

##### Aleatory

In [ ]:
agent.naive_choice(data_preprocessed, signal="Aleatory D-0", policy={"entry": lambda signal: signal > 0.5, "out": lambda signal: signal <= 0.5})

##### SMA

In [ ]:
bigger = - np.inf

for entry in np.linspace(0, 1, 21):
    for out in np.linspace(0, 1, 21):
    
        profit = agent.naive_choice(data_preprocessed, signal="SMA D-0", policy={"entry": lambda signal: signal > entry, "out": lambda signal: signal <= out}, graphic=False)
        # print(f"Profit: {profit:.2f} with Entry: {entry:.2f}, Out: {out:.2f}")
        if profit > bigger:
            bigger = profit
            print(f"\nNew best: {bigger:.2f} with Entry: {entry:.2f}, Out: {out:.2f}\n\n")
            

In [ ]:
profit = agent.naive_choice(data_preprocessed, signal="SMA D-0", policy={"entry": lambda signal: signal > 0, "out": lambda signal: signal <= 0})

In [ ]:
profit = agent.naive_choice(sample, signal="SMA D-0", policy={"entry": lambda signal: signal > 0, "out": lambda signal: signal <= 0})

##### RSI

In [ ]:
bigger = - np.inf

for entry in np.linspace(0, 100, 21):
    for out in np.linspace(0, 100, 21):
    
        profit = agent.naive_choice(data_preprocessed, signal="RSI D-0", policy={"entry": lambda signal: signal < entry, "out": lambda signal: signal > out}, graphic=False)
        # print(f"Profit: {profit:.2f} with Entry: {entry:.2f}, Out: {out:.2f}")
        if profit > bigger:
            bigger = profit
            print(f"\nNew best: {bigger:.2f} with Entry: {entry:.2f}, Out: {out:.2f}\n\n")
            

In [ ]:
agent.naive_choice(data_preprocessed, signal="RSI D-0", policy={"entry": lambda signal: signal < 5, "out": lambda signal: signal > 80})

In [ ]:
agent.naive_choice(data_preprocessed, signal="RSI D-0", policy={"entry": lambda signal: signal < 5, "out": lambda signal: signal > 80})

##### AO

In [ ]:
bigger = - np.inf

for entry in np.linspace(0, 100, 21):
    for out in np.linspace(0, 100, 21):
    
        profit = agent.naive_choice(data_preprocessed, signal="AO D-0", policy={"entry": lambda signal: signal > entry, "out": lambda signal: signal <= out}, graphic=False)
        # print(f"Profit: {profit:.2f} with Entry: {entry:.2f}, Out: {out:.2f}")
        if profit > bigger:
            bigger = profit
            print(f"\nNew best: {bigger:.2f} with Entry: {entry:.2f}, Out: {out:.2f}\n\n")
            

In [ ]:
agent.naive_choice(data_preprocessed, signal="AO D-0", policy={"entry": lambda signal: signal > 0, "out": lambda signal: signal <= 0})

In [ ]:
agent.naive_choice(sample, signal="AO D-0", policy={"entry": lambda signal: signal > 0, "out": lambda signal: signal <= 0})

##### MACD

In [ ]:
agent.naive_choice(data_preprocessed, signal="MACD D-0", policy={"entry": lambda signal: signal > 0, "out": lambda signal: signal <= 0})

##### ADX

In [ ]:
agent.naive_choice(data_preprocessed, signal="ADX D-0", policy={"entry": lambda signal: signal > 0.5, "out": lambda signal: signal <= 0.5})

##### BB

In [ ]:
agent.naive_choice(data_preprocessed, signal="BB D-0", policy={"entry": lambda signal: signal > 0.5, "out": lambda signal: signal <= 0.5})

##### Ichimoku

In [ ]:
agent.naive_choice(data_preprocessed, signal="Ichimoku D-0", policy={"entry": lambda signal: signal > 0.5, "out": lambda signal: signal <= 0})

##### Parabolic SAR

In [ ]:
agent.naive_choice(data_preprocessed, signal="pSAR_ind D-0", policy={"entry": lambda signal: signal > 0, "out": lambda signal: signal <= 0.001})

##### Optimization and Fit

In [ ]:
agent.optimizer(n_trials=5)

In [ ]:
agent.fit(path="../saved")

In [ ]:
agent.feature_importances()

### Real Result

In [ ]:
results = agent.predict(sample, path="../", return_df=True)